In [1]:
# Load packages
import requests 
import pandas as pd 
import time
import re

from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import StaleElementReferenceException
import pandas as pd

In [2]:
url = 'https://www.productreview.com.au/listings/uber-eats?page='  
urls = [url + str(page) + "&sortBy=ratingHighest#reviews-list" for page in range(1, 11)]
print(urls)

['https://www.productreview.com.au/listings/uber-eats?page=1&sortBy=ratingHighest#reviews-list', 'https://www.productreview.com.au/listings/uber-eats?page=2&sortBy=ratingHighest#reviews-list', 'https://www.productreview.com.au/listings/uber-eats?page=3&sortBy=ratingHighest#reviews-list', 'https://www.productreview.com.au/listings/uber-eats?page=4&sortBy=ratingHighest#reviews-list', 'https://www.productreview.com.au/listings/uber-eats?page=5&sortBy=ratingHighest#reviews-list', 'https://www.productreview.com.au/listings/uber-eats?page=6&sortBy=ratingHighest#reviews-list', 'https://www.productreview.com.au/listings/uber-eats?page=7&sortBy=ratingHighest#reviews-list', 'https://www.productreview.com.au/listings/uber-eats?page=8&sortBy=ratingHighest#reviews-list', 'https://www.productreview.com.au/listings/uber-eats?page=9&sortBy=ratingHighest#reviews-list', 'https://www.productreview.com.au/listings/uber-eats?page=10&sortBy=ratingHighest#reviews-list']


In [3]:
resp = requests.get(urls[0])
content = BeautifulSoup(resp.content, 'lxml')
print(content)

<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]--><!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]--><!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js" lang="en-US"> <!--<![endif]-->
<head>
<title>Attention Required! | Cloudflare</title>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="noindex, nofollow" name="robots"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<link href="/cdn-cgi/styles/cf.errors.css" id="cf_styles-css" rel="stylesheet"/>
<!--[if lt IE 9]><link rel="stylesheet" id='cf_styles-ie-css' href="/cdn-cgi/styles/cf.errors.ie.css" /><![endif]-->
<style>body{margin:0;padding:0}</style>
<!--[if gte IE 10]><!-->
<script>
  if (!navigator.cookieEnabled) {
    window.addEventListener('DOMContentLoaded', function () {

In [7]:
def scrape_reviews(urls):
    service = Service(executable_path="C:\\Program Files (x86)\\chromedriver.exe")
    driver = webdriver.Chrome(service=service)
    all_reviews = []

    for url in urls:
        driver.get(url)
        reviews = []
        time.sleep(10)#to avoid captcha

        try:
            review_cards = driver.find_elements(By.CSS_SELECTOR, '.dGNZOq.i8dffl.I93Xxj')

            for card in review_cards:
                try:
                    name = card.find_element(By.CSS_SELECTOR, '.QZg_Tk._1mDXm.pX1OeA.Wb4ihj.Nx0Cq8').text

                    date_element = card.find_element(By.TAG_NAME, 'time')
                    date = date_element.get_attribute('datetime')

                    rating_element = card.find_element(By.CSS_SELECTOR, '.b5bu2E._VTlzY')
                    rating = int(re.search(r'\d+', rating_element.get_attribute('title')).group())

                    title = card.find_element(By.CSS_SELECTOR, 'h3').text

                    review_element = card.find_element(By.CSS_SELECTOR, '.KBnieB.WUOv0Y')
                    review = review_element.text

                    review_data = {
                        'review': review,
                        'date': date,
                        'stars': rating,
                        'name': name,
                        'source': 'productreview',
                        'title': title
                    }

                    reviews.append(review_data)

                except StaleElementReferenceException:
                    # Retry locating the element in case of a stale element reference
                    continue
                except Exception as e:
                    print(e)
                    continue

            all_reviews.extend(reviews)

        except Exception as e:
            print(e)
            continue

    driver.quit()

    return pd.DataFrame(all_reviews)

In [9]:
df_reviews = scrape_reviews(urls)
df_reviews

,review,date,stars,name,source,title
0,PLEASANT UNEXPECTED CUSTOMER SERVICEI gave Ube...,2024-01-17T06:21:07.000Z,5,evelweevil,productreview,PLEASANT UNEXPECTED CUSTOMER SERVICE
1,UberEats or shall be known as UberEats Money i...,2023-12-15T00:15:44.000Z,5,Cruella Devil,productreview,UberEats or shall be known as UberEats Money i...
2,Delivery delaysWe were provided with a final t...,2023-12-09T07:38:46.000Z,5,Car,productreview,Delivery delays
3,Uber eat fast and easyFast and easy\nI would s...,2023-11-26T01:12:39.000Z,5,You,productreview,Uber eat fast and easy
4,Used uber eats for the first timeExcellent com...,2023-10-12T02:28:37.000Z,5,Fabian G.,productreview,Used uber eats for the first time
...,...,...,...,...,...,...
330,MEMBERSHIP CANCELLEDNearly every time I use ub...,2023-12-30T21:22:42.000Z,1,Alison,productreview,MEMBERSHIP CANCELLED
331,"Food not deliveredMy food was not delivered, U...",2023-12-30T20:58:33.000Z,1,LAURA,productreview,Food not delivered
332,Delivering for UberEatsI have worked as a cont...,2023-12-30T04:07:10.000Z,1,Fabrice L,productreview,Delivering for UberEats
333,Uber driver steals foodWouldn't use this compa...,2023-12-27T22:10:45.000Z,1,Roland F.,productreview,Uber driver steals food


In [10]:
df_reviews.to_csv("ubereats_productreview.csv", index= False)